# Format Data SQL

### Loading initial data

In this lesson, we'll begin by formatting our willy wonka data.  Let's get started.

### Initial Formatting - column names and type

First do some basic transformations in pandas -- make sure the column names are all lowercase, and convert the `order_data` column to be a datetime.  Also make sure that any columns that can be numeric are of the dtype float or integer before writing an initial table to the database.

Then save the data to the `willy_wonka` database as a table called `raw_orders`.  

In [4]:
import pandas as pd
from sqlalchemy import create_engine

df = pd.read_csv('solutions_simulation_willy_wonka.csv')
engine = create_engine('postgresql://@localhost:5432/willy_wonka')

def format_data(df):
    df.columns = [col.lower() for col in df.columns]
    df = df.assign(order_date = pd.to_datetime(df['order_date']))
    return df

def insert_data(df, engine):
    df.to_sql('raw_orders', engine, if_exists = 'replace')
    return df


In [5]:
formatted_df = format_data(df)
inserted_df = insert_data(formatted_df, engine)

Check that the data is properly loaded below.

In [6]:
df = pd.read_sql("""select * from raw_orders""", engine)
df[:2]

# 	index	order_id	list_lineitem_id	user_id	order_date	order_value	order_item_count	order_single_item	order_channel	order_coupon_code	...	order_store_city	order_store_id	order_day_part	order_is_freeshipping	list_sku	list_sku_category	list_sku_flavor	user_loyalty_program	user_age	user_gender
# 0	0	TXN-52889719	ITEM-2008965888,ITEM-2009089674	USER-16961214	2021-05-04	49.325488	2	False	digital_direct	express5	...	Dallas	Dallas-3	evening	1	SKU-3014,SKU-3002	Energy Shot,Energy Bar	Chocolate Brownie,Peanut Butter	bronze	49	F
# 1	1	TXN-52889726	ITEM-2009045400,ITEM-2009302964	USER-17030520	2021-03-12	22.993890	2	False	in_store	express5	...	Phoenix	Phoenix-3	evening	1	SKU-5012,SKU-5009	Protein Bar,Protein Powder	Cookies and Cream,Coconut	silver	31	F

,index,order_id,list_lineitem_id,user_id,order_date,order_value,order_item_count,order_single_item,order_channel,order_coupon_code,...,order_store_city,order_store_id,order_day_part,order_is_freeshipping,list_sku,list_sku_category,list_sku_flavor,user_loyalty_program,user_age,user_gender
0,0,TXN-52889719,"ITEM-2008965888,ITEM-2009089674",USER-16961214,2021-05-04,49.325488,2,False,digital_direct,express5,...,Dallas,Dallas-3,evening,1,"SKU-3014,SKU-3002","Energy Shot,Energy Bar","Chocolate Brownie,Peanut Butter",bronze,49,F
1,1,TXN-52889726,"ITEM-2009045400,ITEM-2009302964",USER-17030520,2021-03-12,22.993890,2,False,in_store,express5,...,Phoenix,Phoenix-3,evening,1,"SKU-5012,SKU-5009","Protein Bar,Protein Powder","Cookies and Cream,Coconut",silver,31,F


> We did not have to deal with it here, but if any of our column names had any spaces, we should replace them with underscores.

### Formatting Data

Now if we look at our `raw_orders` table, there is still more work to do.  First, we should extract the `order_date` column into separate columns like the month, week, and day_of_week  from the orders.  Save the resulting query into a pandas dataframe, and then we can store the dataframe as a new table called orders.

In [7]:
# df.columns

query = """select order_id, user_id, list_sku,
       list_sku_category, list_sku_flavor, list_lineitem_id,
       order_date, order_value, order_item_count, order_single_item,
       order_channel, order_coupon_code, order_referrer,
       order_ad_shown, order_store_city, order_store_id,
       order_day_part, order_is_freeshipping, user_loyalty_program,
       user_age, user_gender,
       EXTRACT(month FROM order_date) as month, EXTRACT(week FROM order_date) as week, 
EXTRACT(DOW FROM order_date) as day_of_week from raw_orders"""

df_with_date = pd.read_sql(query, engine)
df_with_date.columns

Index(['order_id', 'user_id', 'list_sku', 'list_sku_category',
       'list_sku_flavor', 'list_lineitem_id', 'order_date', 'order_value',
       'order_item_count', 'order_single_item', 'order_channel',
       'order_coupon_code', 'order_referrer', 'order_ad_shown',
       'order_store_city', 'order_store_id', 'order_day_part',
       'order_is_freeshipping', 'user_loyalty_program', 'user_age',
       'user_gender', 'month', 'week', 'day_of_week'],
      dtype='object')

In [8]:
df_with_date.to_sql('orders', engine, if_exists = 'replace')

625

### The tricky part

Now let's take another look at our data.

In [9]:
df_with_date[:2]

,order_id,user_id,list_sku,list_sku_category,list_sku_flavor,list_lineitem_id,order_date,order_value,order_item_count,order_single_item,...,order_store_city,order_store_id,order_day_part,order_is_freeshipping,user_loyalty_program,user_age,user_gender,month,week,day_of_week
0,TXN-52889719,USER-16961214,"SKU-3014,SKU-3002","Energy Shot,Energy Bar","Chocolate Brownie,Peanut Butter","ITEM-2008965888,ITEM-2009089674",2021-05-04,49.325488,2,False,...,Dallas,Dallas-3,evening,1,bronze,49,F,5.0,18.0,2.0
1,TXN-52889726,USER-17030520,"SKU-5012,SKU-5009","Protein Bar,Protein Powder","Cookies and Cream,Coconut","ITEM-2009045400,ITEM-2009302964",2021-03-12,22.993890,2,False,...,Phoenix,Phoenix-3,evening,1,silver,31,F,3.0,10.0,5.0


We can see that a single column like the `list_sku` has multiple entries -- for the two products purchased in that order, and we have the same issues with the respective `list_sku_category`, `list_sku_flavor`, and `list_lineitem_id` columns.  So a single order can often have two purchases.  We want to separate the these purchases into separate rows, so that we can query them.

You can see a sample of the result here.  Notice that the first two rows have the same `order_id` but different `list_sku` values.

> <img src="./line-item.png" width="90%">

You can also see the desired results here.

In [10]:
df_line_items = pd.read_csv('./raw_line_items.csv', index_col = 0)
df_line_items[:3]

,index,order_id,user_id,list_sku,list_sku_category,list_sku_flavor,list_lineitem_id,order_date,month,week,...,order_coupon_code,order_referrer,order_ad_shown,order_store_city,order_store_id,order_day_part,order_is_freeshipping,user_loyalty_program,user_age,user_gender
0,0,TXN-52889719,USER-16961214,SKU-3002,Energy Bar,Peanut Butter,ITEM-2009089674,2021-05-04,5.0,18.0,...,express5,Microsoft,week40_10,Dallas,Dallas-3,evening,1,bronze,49,F
1,1,TXN-52889719,USER-16961214,SKU-3014,Energy Shot,Chocolate Brownie,ITEM-2008965888,2021-05-04,5.0,18.0,...,express5,Microsoft,week40_10,Dallas,Dallas-3,evening,1,bronze,49,F
2,2,TXN-52889726,USER-17030520,SKU-5009,Protein Powder,Coconut,ITEM-2009302964,2021-03-12,3.0,10.0,...,express5,Google,week35_ 1,Phoenix,Phoenix-3,evening,1,silver,31,F


Ok, so now write the query -- completely in SQL -- to produce the desired results (seen in the CSV file above).

In [13]:
query = """with first_sku as (
    select order_id, user_id, split_part(list_sku, ',', 1) as list_sku,
	split_part(list_sku_category, ',', 1) as list_sku_category,
     split_part(list_sku_flavor, ',', 1) as list_sku_flavor, 
     split_part(list_lineitem_id, ',', 1) as list_lineitem_id,
     order_date, month, week, day_of_week,
       order_value, order_item_count, order_single_item, order_channel,
       order_coupon_code, order_referrer, order_ad_shown,
       order_store_city, order_store_id, order_day_part,
       order_is_freeshipping, 
       user_loyalty_program, user_age, user_gender
     from orders
     ), second_sku as (
    select order_id, user_id, split_part(list_sku, ',', 2) as list_sku,
	split_part(list_sku_category, ',', 2) as list_sku_category,
     split_part(list_sku_flavor, ',', 2) as list_sku_flavor, 
     split_part(list_lineitem_id, ',', 2) as list_lineitem_id,
     order_date, month, week, day_of_week,
       order_value, order_item_count, order_single_item, order_channel,
       order_coupon_code, order_referrer, order_ad_shown,
       order_store_city, order_store_id, order_day_part,
       order_is_freeshipping, 
       user_loyalty_program, user_age, user_gender
     from orders 
     )
     select * from first_sku union select * from second_sku where list_sku != '' order by order_id asc 
     """

coerced_line_items_df = pd.read_sql(query, engine)


> Then we can save the result to a table called `raw_lineitems`.

In [19]:
coerced_line_items_df.to_sql('raw_lineitems', engine, if_exists = 'replace', index = False)

27

> You can check that your results are correct below.

In [20]:
raw_lineitems_df = pd.read_sql('select * from raw_lineitems', engine)
raw_lineitems_df[:5]
# 	index	list_lineitem_id	order_id	user_id	order_date	list_sku	list_sku_category	order_item_count	order_single_item	order_channel	...	order_store_id	order_day_part	order_is_freeshipping	user_loyalty_program	user_age	user_gender	list_sku_flavor	month	week	dow
# 0	0	ITEM-2009089674	TXN-52889719	USER-16961214	2021-05-04	SKU-3002	Energy Bar	ITEM-2009089674	False	digital_direct	...	Dallas-3	evening	1	bronze	49	F	Peanut Butter	5.0	18.0	2.0
# 1	1	ITEM-2008965888	TXN-52889719	USER-16961214	2021-05-04	SKU-3014	Energy Shot	ITEM-2008965888	False	digital_direct	...	Dallas-3	evening	1	bronze	49	F	Chocolate Brownie	5.0	18.0	2.0

,order_id,user_id,list_sku,list_sku_category,list_sku_flavor,list_lineitem_id,order_date,month,week,day_of_week,...,order_coupon_code,order_referrer,order_ad_shown,order_store_city,order_store_id,order_day_part,order_is_freeshipping,user_loyalty_program,user_age,user_gender
0,TXN-52889719,USER-16961214,SKU-3002,Energy Bar,Peanut Butter,ITEM-2009089674,2021-05-04,5.0,18.0,2.0,...,express5,Microsoft,week40_10,Dallas,Dallas-3,evening,1,bronze,49,F
1,TXN-52889719,USER-16961214,SKU-3014,Energy Shot,Chocolate Brownie,ITEM-2008965888,2021-05-04,5.0,18.0,2.0,...,express5,Microsoft,week40_10,Dallas,Dallas-3,evening,1,bronze,49,F
2,TXN-52889726,USER-17030520,SKU-5009,Protein Powder,Coconut,ITEM-2009302964,2021-03-12,3.0,10.0,5.0,...,express5,Google,week35_ 1,Phoenix,Phoenix-3,evening,1,silver,31,F
3,TXN-52889726,USER-17030520,SKU-5012,Protein Bar,Cookies and Cream,ITEM-2009045400,2021-03-12,3.0,10.0,5.0,...,express5,Google,week35_ 1,Phoenix,Phoenix-3,evening,1,silver,31,F
4,TXN-52889727,USER-17206628,SKU-3003,Energy Drink,Chia Crunch,ITEM-2009244633,2021-03-27,3.0,12.0,6.0,...,express5,Yahoo!,week 2_ 2,San Jose,SanJose-3,evening,0,none,27,M


### Summary

In this lesson we worked with formatting our data.  We began by doing some initial formatting in pandas -- just changing the column names to be lowercased, removing spaces (if they exist), and changing columns to be datetime or integers where possible.  

After that initial cleanup, we then used sql queries to extract week, month, and day of week data -- which can be valuable for performing our group bys later on.  

And finally, we flattened the data that was nested under one column -- the `list_sku`, `list_sku_category`, `list_sku_flavor`, and `list_lineitem_id` columns.